In [2]:
# !pip install netCDF4
# !pip install cartopy
# !pip install basemap 



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
  Using cached Cartopy-0.21.1.tar.gz (10.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for cartopy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [292 lines of output]
      <string>:90: UserWarning: Unable to determine GEOS version. Ensure you have 3.7.2 or later installed, or installation may fail.
      WARNING setuptools_scm.pyproject_reading toml section missing 'pyproject.toml does not contain a tool.setuptools_scm section'
      Traceback (most recent call last):
        File "/tmp/pip-build-env-8d1b_tye/overlay/lib/python3.8/site-packages/setuptools_scm/_integration/pyproject_reading.py", line 36, in read_pyproject
          section = defn.get("tool", {})[tool_name]
      KeyError: 'setu

In [2]:
import netCDF4 as nc
import os
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [3]:
import tqdm

In [4]:
# Constructing a relative path to move one directory backward
data_relative_path = os.path.join('..', 'Data','CMIP6')
precipitation_path = os.path.join(data_relative_path,'Precipitation')
WTIO_path = os.path.join(data_relative_path,'WTIO')
SEIO_path = os.path.join(data_relative_path,'SEIO')

In [5]:
file_path = "pr_Amon_ACCESS-CM2_historical_r1i1p1f1_gn_19811016-20101016_v20191108.nc"
data = nc.Dataset(f"{precipitation_path}/{file_path}")


# Initial data Analysis and Presentation

First, we select the latitude and longitude range for the Middle East (25°N to 40°N, 25°E to 40°E)

In [ ]:
for file in tqdm.tqdm(os.listdir(precipitation_path)):
    filename = os.fsdecode(file)
    path = f"{precipitation_path}/{filename}"
    data = nc.Dataset(path)
    # print(filename)
    # print(ds.variables['pr'][:].shape)
    lat_name = 'lat' if 'lat' in data.variables else 'latitude'
    lon_name = 'lon' if 'lon' in data.variables else 'longitude'
    lat = data.variables[f'{lat_name}'][(data.variables[f'{lat_name}'][:] >= 25) & (data.variables[f'{lat_name}'][:] <= 40)]
    lon = data.variables[f'{lon_name}'][(data.variables[f'{lon_name}'][:] >= 25) & (data.variables[f'{lon_name}'][:] <= 40)]
    precipitation = data.variables['pr'][:]
    avg_pr_middle_east = np.mean(precipitation,axis=0)
    convert_unit_to_mm_transform = (86400*31) 
    mm_avg_lat_lon = avg_pr_middle_east * convert_unit_to_mm_transform 
    # Plot the map with the heatmap overlay
    plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([min(lat), max(lat),min(lon),max(lon)], crs=ccrs.PlateCarree())
    
    # Add map features
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAND, color='lightgray')
    
    plt.imshow(mm_avg_lat_lon, origin='lower', cmap='jet', extent=[lon[0], lon[-1], lat[0], lat[-1]],
               transform=ccrs.PlateCarree(), interpolation='nearest')
    
    # Add colorbar
    cbar = plt.colorbar(label='Average Precipitation (mm/month)', orientation='horizontal')
    cbar.set_label('Average Precipitation (mm/month)')
    plt.title(f'Avg Prec. for October, {filename}')
    
    plt.show()


  0%|          | 0/24 [00:00<?, ?it/s]

In [6]:
lat = data.variables['lat'][(data.variables['lat'][:] >= 25) & (data.variables['lat'][:] <= 40)]
lon = data.variables['lon'][(data.variables['lon'][:] >= 25) & (data.variables['lon'][:] <= 40)]

Next, we extract the precipitation data for the Middle East region and average it over the 30-year period.

In [7]:
precipitation = data.variables['pr'][:]
avg_pr_middle_east = np.mean(precipitation,axis=0)

An important step - we need to convert the precipitation units from kg m-2 s-1 to mm/month. To do so, we multiply the precipitation values by the number of seconds in a day (86400) and the number of days in a month (31). Along the way we multiply and divide by 1000 to convert the units from kg to g and then to mm.

In [8]:
convert_unit_to_mm_transform = (86400*31) 
mm_avg_lat_lon = avg_pr_middle_east * convert_unit_to_mm_transform 

Finally, we plot the average precipitation for the Middle East region for the month of October.

In [ ]:
# Plot the map with the heatmap overlay
plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([min(lat), max(lat),min(lon),max(lon)], crs=ccrs.PlateCarree())

# Add map features
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND, color='lightgray')

plt.imshow(mm_avg_lat_lon, origin='lower', cmap='jet', extent=[lon[0], lon[-1], lat[0], lat[-1]],
           transform=ccrs.PlateCarree(), interpolation='nearest')

# Add colorbar
cbar = plt.colorbar(label='Average Precipitation (mm/month)', orientation='horizontal')
cbar.set_label('Average Precipitation (mm/month)')
plt.title('Average Precipitation for October in the Middle East')

plt.show()
